# daschlab Tutorial: Handling Source Splitting in HD 5501

HD 5501 is a 9th-mag eclipsing binary with a period of around 7.5 days. In DASCH's APASS calibration, it is subject to
the ["source splitting" Known Issue][sski], which causes its data to be split among several different raw lightcurves.
This notebook demonstrates how to merge the split data.

[sski]: https://dasch.cfa.harvard.edu/dr7/ki/source-splitting/

---

## **Important:** Non-introductory tutorial

*daschlab* is **interactive** software and it's very difficult to express some of the key steps in writing.

#### **If you're unfamiliar with *daschlab*,** complete [the RY Cnc introductory tutorial][rycnc] before undertaking this one.

[rycnc]: https://dasch.cfa.harvard.edu/dr7/rycnc/

---

## Preliminaries

Here we'll do some standard imports.

In [ ]:
# Get the main module:
import daschlab

# Set up Bokeh plots:
from bokeh.io import output_notebook
output_notebook()

# Get some other imports that will be useful:
from astropy import units as u
from bokeh.plotting import figure, show
import numpy as np
from daschlab.photometry import AFlags, BFlags

---

## Set up the session

Now we configure our analysis session: where the data are stored, what target we're interested in, and which photometric reference catalog to use.
The configuration here is completely standard.

In [ ]:
SOURCE = "HD 5501"

sess = daschlab.open_session(source=SOURCE)
sess.select_target(name=SOURCE)
sess.select_refcat("apass")

---

## Connect to WWT and display the "refcat" sources

We can use daschlab's WWT integration to display the catalog sources of interest on the sky.
This is always a good first step when digging into the data.

#### **Before proceeding, make sure that the WWT JupyterLab app is open** — the [RY Cnc slideshow][slides] shows what to do

Once that's done, we can proceed:

[slides]: https://dasch.cfa.harvard.edu/dr7/rycnc/#/7

In [ ]:
await sess.connect_to_wwt()

In [ ]:
# Display the reference catalog in WWT
sess.refcat().show()

You might notice an astrometric offset between the catalog and basemap imagery. It's an unfortunate feature of the default WWT DSS mosaic. You can fix it by changing the WWT background to **PanSTARRS1 3pi**.

---

## Display a nice sample cutout

As usual, a nice next "warmup" step is pull up a high-quality DASCH cutout around our target. *daschlab* has 
builtin functionality to suggest candidates.

In [ ]:
# Print mini table of candidates
sess.exposures().candidate_nice_cutouts()

In [ ]:
# This exp_local_id looks good ...
sess.exposures().show(8732)

---

## Identify that "source splitting" is an issue

To analyze the lightcurve of this object, the natural next step is to look at the lightcurve of "target 0", the 
entry in our source catalog that is spatially closest to HD 5501's catalog position:

In [ ]:
lc = sess.lightcurve(0)
lc.summary()

This is immediately suspicious. First of all, HD 5501's B magnitude should be more like 9.0, not 9.9.
Second, a star this bright should have thousands of detections, not hundreds.

In the DASCH data set, these kinds of things are evidence that [source splitting][ss] is potentially coming into play.

[ss]: https://dasch.cfa.harvard.edu/dr7/ki/source-splitting/

In order to investigate this, let's summarize the lightcurve for "target 1", the catalog entry that is second-closest to our target.

In [ ]:
sess.lightcurve(1).summary()

The mean magnitude is about the same, and once again there are far fewer detections than there should be for
a star that is actually this magnitude. This supports the idea that detections of HD 5501 are getting split
between different catalog sources.

To double-check this, let's print out the catalog information associated with this catalog entry:

In [ ]:
sess.refcat()[1]

The source name (`ref_text`) looks like `N{numbers...}` and the cataloged magnitude (`stdmag`) is undefined, indicating that this is a source from the Guide Star Catalog v2.3.2 without a magnitude in the blue band. If this were truly a 9.5-mag star, it would absolutely be in the catalog already. This is yet more evidence that source-splitting is at play.

How about the next source?

In [ ]:
sess.lightcurve(2).summary()

This entry has many thousands of detections, and a magnitude much closer to what we expect. It
looks like the bulk of the detections got associated with this catalog entry.

Let's keep on checking more sources.

In [ ]:
sess.lightcurve(3).summary()

There are fewer detections, but the mean magnitude is still around that of our target.

OK, how far are we going to have to go here? It can be helpful to take a look at the reference
catalog to get a sense of the characteristics of the sources in question:

In [ ]:
# Get the catalog table
rc = sess.refcat()

# Add a column quantifying the distance between the catalog entry and the star of interest
rc["sep"] = sess.query().pos_as_skycoord().separation(rc["pos"]).arcsec

# Print out key characteristics of the 20 nearest sources
rc[:20][["local_id", "ref_text", "sep", "stdmag"]]

The 17th-nearest catalog source (the one with `local_id = 16`) is finally another official catalog APASS source.
As a rough rule of thumb, everything nearer to our target than this is a candidate to have been assigned
some detections of HD 5501. Note that HD 5501 itself has a missing catalog magnitude (`stdmag`) — this is
probably why the source splitting is so severe here.

To visualize the reference catalog better, it helps to increase the `size_vmin` setting of the WorldWide
Telescope table display "layer". This number controls the relationship between the catalog magnitude
(`stdmag`) and circle radius; increasing it causes the faintest sources to be drawn with bigger circles.
We can edit this parameter using the WWT table Python object, obtainable by calling `refcat.show()` again:

In [ ]:
sess.refcat().show().size_vmin += 2

To keep things tractable, let's just declare that we're only going to look for source-split measurements
in the lightcurves of catalog sources that are closer than 40 arcsec to the true source position.
Looking at the above table, this means that we'll look at the eleven nearest sources.

In [ ]:
N_TO_MERGE = 11

Let's download the data for all of these lightcurves. This will take a while — time to go get a cup of coffee.

In [ ]:
lcs = [sess.lightcurve(i) for i in range(N_TO_MERGE)]

Let's print out a summary of what we've got:

In [ ]:
for i, lc in enumerate(lcs):
    print("***** lightcurve index", i)
    lc.summary()
    print()

This seems like a decent starting point for the merger analysis.

---

## Merge the lightcurves

Before we merge, let's apply standard rejections to all of the individual lightcurves:

In [ ]:
for lc in lcs:
    lc.apply_standard_rejections()
    print()

This flags out a lot of data.

We're now ready to merge. We just need to call a library function:

In [ ]:
import daschlab.lightcurves

# this will take a moment to run:
mlc = daschlab.lightcurves.merge(lcs)
mlc.summary()

This looks promising. The mean magnitude is around what it should be, and we have around 2400 non-rejected detections.
In comparison, source #2 had around 2000 good detections after applying standard rejections, so by merging we've
increased the amount of available data by about 20%.

The merge algorithm looks at every detection in every input lightcurve, and groups them by the DASCH exposure where
they originated. When source splitting is in play, in most cases there will be exactly one lightcurve with a detection
for each exposure — the detection just jumps among sources based on things like small astrometric errors.

The resulting lightcurve gains a column named `merge_n_hits` that tells us how many lightcurves had detections for
the each exposure. If more than one lightcurve has a hit, the merge algorithm chooses the detection with the
magnitude value closest to the mean magnitude, since we can't fully trust the astrometry.

Let's make a little histogram of how many lightcurve points have how many hits:

In [ ]:
n_hit_counts = {}

for row in mlc:
    nh = row["merge_n_hits"]
    n_hit_counts[nh] = n_hit_counts.get(nh, 0) + 1

for n_hits, n_rows in sorted(n_hit_counts.items()):
    print(f"- there were {n_rows:4} rows with {n_hits:4} hits in the source lightcurves")

This is very promising. There's onle one case where there was any ambiguity.
To be safe, let's reject it:

In [ ]:
mlc.reject.where(mlc["merge_n_hits"] > 1, tag="merge_ambiguity")

---

## Examine the merged result

Enough dilly-dallying, let's look at what we got!

In [ ]:
mlc.plot()

There are a couple of clearly bogus low points, and also a number of bogus limits. But overall,
we appear to have a lot of reasonable-looking measurements.

Let's plot the data at the known orbital phase to check how things look.

In [ ]:
PERIOD = 7.5352272 # days

# The offset here gets the eclipse to show up around ph ~ 0.5:
mlc['ph'] = (mlc['time'].jd / PERIOD - 0.4) % 1

mlc.plot(x_axis="ph")

We can definitely see an eclipse there!

---

## Filter data: Incorrect merges

The most obvious issue with the merged lightcurve is the very low points. Let's examine one of them, identifying its "local ID"
by mousing over one of the points in the plot above.

In [ ]:
mlc[1600]

At the far right of the table row printout, there is a column called `source_lc_index` identifying which
source lightcurve this row came from. You should see that these rows all come from source curve #8.
You might have noticed above that the `stdmag` for this catalog source is *relatively* bright:

In [ ]:
sess.refcat()[8]["stdmag"]

This is, in fact, in the ballpark of the detections that we're getting. So these are clearly mis-identifications
in our lightcurve. We can flag them:

In [ ]:
mlc.reject.where(
    (mlc["source_lc_index"] == 8) & (mlc["magcal_magdep"] > 13 * u.mag), 
    tag="merge_error"
)

We also clearly have a lot of spurious upper limits. For certain science applications, you might need to dig
into those and understand where there coming from. Here, we just want a nice phase plot, so let's just throw
them away:

In [ ]:
mlc.reject_unless.detected(tag="ignore_limits")

In [ ]:
# How do things look now?
mlc.plot(x_axis="ph")

There's clearly more flagging that could be done, but this tutorial will stop here.